In [5]:
import pandas as pd

In [18]:
data = pd.read_csv('../data/twitter_davinci_ds.csv', error_bad_lines=False, lineterminator='\n')
data.sample(5)

,Unnamed: 0,LangID,TopicID,HateLabel,TweetText
10918,10918,1.0,3.0,0.0,RICH NOYES Rewind: The last four years showed ...
26445,26445,1.0,2.0,2.0,"The assassination of Iranian General, Qassim S..."
19764,19764,1.0,3.0,0.0,Mississippi no longer has a state flag until v...
9320,9320,1.0,1.0,0.0,Thankfully now I live in Country that hate spe...
17409,17409,1.0,3.0,0.0,Warren Accuses Trump of ‘Bungling Every Aspect...


### Code to Generate More Tweets

In [3]:
import os
import openai

In [ ]:
openai.api_key = 'insert API key'
def get_hate(text):

    response = openai.Completion.create(
    model="text-davinci-002",
    prompt=f"{text}",
    )

    return response['choices'][0]['text']  

In [1]:
from multiprocessing import Pool, cpu_count

In [ ]:
offensive_tweets = df.loc[df.HateLabel == 1].iloc[:100]

def process_row(row):
    old_hate = row['TweetText']
    new_hate = get_hate(f'Give me similar texts to this one but use other hate words: "{old_hate}"')

    # Create new row
    new_row = {'TweetID': 'DAV_2_TweetID',  # Add appropriate TweetID
               'LangID': row['LangID'],
               'TopicID': row['TopicID'],
               'HateLabel': row['HateLabel'],
               'TweetText': new_hate} 

    return new_row

offensive_tweets = df.loc[df.HateLabel == 1]


with Pool(cpu_count()) as p:
    new_rows = p.map(process_row, offensive_tweets.to_dict('records'))

new_df = pd.DataFrame(new_rows)

offensive_tweets = pd.concat([offensive_tweets, new_df], ignore_index=True)
offensive_tweets.to_csv('gen_off_texts.csv')
    